**Importing Packages**

In [2]:
#importing required libaries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re 
import string

import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

#keras
from keras import *
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.optimizers import  Adam
from keras import regularizers

**Malayalam Dataset**

In [3]:
#reading the Malayalam dataset
train=pd.read_csv('/content/drive/MyDrive/offensive language/Malayalam dataset/Mal_Training_data.tsv',sep='\t', index_col=[0])
test=pd.read_csv('/content/drive/MyDrive/offensive language/Malayalam dataset/mal_test_data_with_labels.tsv',sep='\t', index_col=[0])

# **Removing punctuation**

In [4]:
import string
def remove_punctuations(txt):
    text_nopunc="".join([c for c in txt if c not in string.punctuation])
    return text_nopunc

train['Text']=train['Text'].apply(lambda x: remove_punctuations(x))
train

,Text,Category
MA_YT001,Thaankal enthaan cheyyarullath😛,NOT
MA_YT002,Ee theetam WCC feminichigalude news aarkk vena...,OFF
MA_YT003,fukru nem tiktok oolakale vilich charcha nadat...,OFF
MA_YT004,Aashiq abu produce cheytharunnel ee problems u...,NOT
MA_YT005,Pennungal oru team aayal ath moonjum ennu epoo...,OFF
...,...,...
MA_YT3996,Eee parasayam thanne thettanu Ella achanmaraya...,NOT
MA_YT3997,Ente bagathum thetundh ee vazhikke veraan paad...,NOT
MA_YT3998,Kuttiye njan kettikolaam swarnam onnum venda e...,NOT
MA_YT3999,Chumma veettil irunna chakkiye trollanmaarkku ...,NOT


# **Spliting to Dev Dataset**

In [5]:
X_train, X_dev, y_train, y_dev = train_test_split(train['Text'], train['Category'], test_size=0.30, random_state=42)

X_test= test['Text']
y_test= test['Category']

# **Encoding**

In [6]:
Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.transform(y_test)
y_dev = Encoder.transform(y_dev)

# **Long Short Term Memory(LSTM)**

In [7]:
from keras.preprocessing.text import one_hot

#use onehot in train
voc_size = 1000

train_onehot = [one_hot(words, voc_size)for words in X_train]
dev_onehot = [one_hot(words, voc_size)for words in X_dev]
test_onehot = [one_hot(words, voc_size)for words in X_test]

In [8]:
#performing pad_sequences
from keras.preprocessing.sequence import pad_sequences

sent_length=100
X_train=pad_sequences(train_onehot,padding='pre',maxlen=sent_length)
X_dev=pad_sequences(dev_onehot,padding='pre',maxlen=sent_length)
X_test = pad_sequences(test_onehot,padding='pre',maxlen=sent_length)

In [9]:
dim=40
model=Sequential()

#embedding layer
model.add(Embedding(voc_size,dim,input_length=sent_length))

#input layer
model.add(LSTM(1000, input_shape=(1000,1), return_sequences=False))

#hidded layer
model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01) ))

#output layer
model.add(Dense(5, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])

#model.compile('adam','mse')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [10]:
#summary of LSTM model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 40)           40000     
_________________________________________________________________
lstm (LSTM)                  (None, 1000)              4164000   
_________________________________________________________________
dense (Dense)                (None, 500)               500500    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2505      
Total params: 4,707,005
Trainable params: 4,707,005
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(X_train, y_train, 
                    batch_size=64,
                    epochs=10, validation_data=(X_dev, y_dev)                 
                    )

Epoch 1/10
44/44 [==============================] - 32s 186ms/step - loss: 5.9846 - accuracy: 0.4646 - val_loss: 2.7331 - val_accuracy: 0.4925
Epoch 2/10
44/44 [==============================] - 7s 161ms/step - loss: 2.4377 - accuracy: 0.5218 - val_loss: 1.9005 - val_accuracy: 0.5075
Epoch 3/10
44/44 [==============================] - 7s 161ms/step - loss: 1.7522 - accuracy: 0.5184 - val_loss: 1.4743 - val_accuracy: 0.5033
Epoch 4/10
44/44 [==============================] - 7s 161ms/step - loss: 1.3550 - accuracy: 0.6475 - val_loss: 1.2832 - val_accuracy: 0.5275
Epoch 5/10
44/44 [==============================] - 7s 162ms/step - loss: 1.0749 - accuracy: 0.6927 - val_loss: 1.2100 - val_accuracy: 0.5017
Epoch 6/10
44/44 [==============================] - 7s 163ms/step - loss: 0.9189 - accuracy: 0.6988 - val_loss: 1.1415 - val_accuracy: 0.5117
Epoch 7/10
44/44 [==============================] - 7s 164ms/step - loss: 0.7721 - accuracy: 0.7248 - val_loss: 1.0988 - val_accuracy: 0.5317
Epoch

In [12]:
#classified with test set
y_pred_test_LSTM = model.predict(X_test)

In [13]:
from sklearn import metrics

pred_LSTM = model.predict(X_test)
#predictions = np.around(predictions)
y_test_non_category = y_test
y_predict_non_category = [ np.argmax(t) for t in pred_LSTM ]

In [14]:
#classification report
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test_non_category, y_predict_non_category)) 
print(classification_report(y_test_non_category, y_predict_non_category, zero_division=0))

[[251 222]
 [251 227]]
              precision    recall  f1-score   support

           0       0.50      0.53      0.51       473
           1       0.51      0.47      0.49       478

    accuracy                           0.50       951
   macro avg       0.50      0.50      0.50       951
weighted avg       0.50      0.50      0.50       951



# **MLP**



In [15]:
#Simple Neural network
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initialising
MLP = Sequential()

# Adding input layer and the first hidden layer
MLP.add(Dense(units = len(train.Category.value_counts()), kernel_initializer = 'uniform', activation = 'relu', input_dim = sent_length))

# Adding second hidden layer
MLP.add(Dense(units = len(train.Category.value_counts()), kernel_initializer = 'uniform', activation = 'relu'))

# Adding output layer
MLP.add(Dense(units = 2, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
MLP.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
MLP.fit(X_train, y_train, batch_size =50 , epochs = 10)

# Predicting the Test set results
y_pred = MLP.predict(X_test)

Epoch 1/10
56/56 [==============================] - 1s 2ms/step - loss: 0.6931 - accuracy: 0.5148
Epoch 2/10
56/56 [==============================] - 0s 2ms/step - loss: 0.6926 - accuracy: 0.5251
Epoch 3/10
56/56 [==============================] - 0s 2ms/step - loss: 0.6920 - accuracy: 0.5164
Epoch 4/10
56/56 [==============================] - 0s 2ms/step - loss: 0.6932 - accuracy: 0.4993
Epoch 5/10
56/56 [==============================] - 0s 2ms/step - loss: 0.6915 - accuracy: 0.5158
Epoch 6/10
56/56 [==============================] - 0s 2ms/step - loss: 0.6911 - accuracy: 0.5199
Epoch 7/10
56/56 [==============================] - 0s 3ms/step - loss: 0.6897 - accuracy: 0.5370
Epoch 8/10
56/56 [==============================] - 0s 3ms/step - loss: 0.6885 - accuracy: 0.5425
Epoch 9/10
56/56 [==============================] - 0s 2ms/step - loss: 0.6907 - accuracy: 0.5280
Epoch 10/10
56/56 [==============================] - 0s 2ms/step - loss: 0.6877 - accuracy: 0.5345


In [16]:
from sklearn import metrics

pred_MLP = MLP.predict(X_test)

y_test_non_category = y_test
y_predict_non_category = [ np.argmax(t) for t in pred_MLP ]

In [17]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test_non_category, y_predict_non_category)) 
print(classification_report(y_test_non_category, y_predict_non_category, zero_division=0))

[[396  77]
 [405  73]]
              precision    recall  f1-score   support

           0       0.49      0.84      0.62       473
           1       0.49      0.15      0.23       478

    accuracy                           0.49       951
   macro avg       0.49      0.49      0.43       951
weighted avg       0.49      0.49      0.43       951

